In [1]:
#Regular modules
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import sys
import math
import datetime
import random
from enum import Enum

#My modules
from dnds_func import calculate_ns, seq_ns
from aa_chemical_properties import aa_charge, aa_charge_dict, aa_functional_group, aa_functional_group_dict, hindex_Kyte_Doolitle, aa_propensity,\
                                    propensity_chou_fasman, aa_volume_group, aa_volume, aa_volume_group_dict, aa_h_bond_donor, aa_h_bond_acceptor
from ext_predictors_codes import sift_codes, polyphen_codes, clinvar_codes
sys.path.append('/home/anat/Research/ExAC/5.HMM_alter_align') 
from calc_exac_freq_func import codon_table
from entropy_func import SE_hist, JSD_background, JSD_hist
sys.path.append('/home/anat/Research/ExAC/8.Whole_domain_analysis')
from go_groups import go_term_group

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
curr_dir = !pwd
domains_th = "10"
SIFT_THRESHOLD = 0.05

#Rare SNP thresholds
MAFT_5 =  0.005
MAFT_05 = 0.0005
MAFT_005 = 0.00005
        
hmm_filename = curr_dir[0]+"/../2.parse_Pfam/v30/domains_hmm_prob_dict.pik"
pfam_aa_order = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
amino_acids_sym = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', "*"]

In [3]:
input_path = curr_dir[0]+"/../6.Ext_features_to_dicts/ext_features_dicts/pfam-v30/"

#Reading the BLOSUM62 dict
with open(curr_dir[0]+"/../BLOSUM62/BLOSUM62_dict.pik", 'rb') as handle:
    blosum62_dict = pickle.load(handle)

#Reading the PAM40 dict
with open(curr_dir[0]+"/../PAM40/PAM40_dict.pik", 'rb') as handle:
    pam40_dict = pickle.load(handle)

#Read binding scores
with open(curr_dir[0]+"/../binding_score/binding_dicts/domains_binding_dict.pik", 'rb') as handle:
    binding_scores_dict = pickle.load(handle) #TODO: is this needed?
with open(curr_dir[0]+"/../binding_score/binding_dicts/domains_all_binding_dict.pik", 'rb') as handle:
    binding_all_scores_dict = pickle.load(handle)
    
#Read binding scores per ligand dictionaries
with open(curr_dir[0]+"/../binding_score/binding_dicts/dna_binding_dict.pik", 'rb') as handle:
    dna_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/rna_binding_dict.pik", 'rb') as handle:
    rna_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/dnabase_binding_dict.pik", 'rb') as handle:
    dnabase_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/rnabase_binding_dict.pik", 'rb') as handle:
    rnabase_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/dnabackbone_binding_dict.pik", 'rb') as handle:
    dnabackbone_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/rnabackbone_binding_dict.pik", 'rb') as handle:
    rnabackbone_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/ion_binding_dict.pik", 'rb') as handle:
    ion_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/peptide_binding_dict.pik", 'rb') as handle:
    peptide_binding_scores_dict = pickle.load(handle)
with open(curr_dir[0]+"/../binding_score/binding_dicts/metabolite_binding_dict.pik", 'rb') as handle:
    metabolite_binding_scores_dict = pickle.load(handle)
    
#domain-sequence dict
with open(curr_dir[0]+"/../5.domains_stats/domains_sequences_dict.pik", 'rb') as handle:
    domain_gene_seq_dict = pickle.load(handle)
    
#Read the list of domains
with open(curr_dir[0]+"/../5.domains_stats/filtered"+domains_th+"_list.pik", 'rb') as handle:
    filtered_domains_list = pickle.load(handle)
filtered_domains_list.sort()

#Read the substitutions table (for the dN/dS calculation)
with open(curr_dir[0]+"/codon_ns_table.pik", 'rb') as handle:
    codon_ns_table = pickle.load(handle)
    
#Read the whole-domain conservation dict
with open(curr_dir[0]+"/../8.Whole_domain_analysis/domain_conservation_dict.pik", 'rb') as handle:
    whole_domain_con_dict = pickle.load(handle)
    
#Read the whole_domain GO classification dict
with open(curr_dir[0]+"/../8.Whole_domain_analysis/domain_go_dict.pik", 'rb') as handle:
    whole_domain_GO_dict = pickle.load(handle)

#Open the HMM dict - takes some time
with open(hmm_filename, 'rb') as handle:
    hmm_prob_dict = pickle.load(handle)    
    
#Creating a list of the intersection of domains with binding scores and domains with states dicts
domains = []
for domain in filtered_domains_list:
    if (domain in binding_all_scores_dict.keys()):
        domains.append(domain)
print "number of domains = "+str(len(domains))

number of domains = 417


In [4]:
#Open the HMM dict
with open(hmm_filename, 'rb') as handle:
    hmm_prob_dict = pickle.load(handle)

In [ ]:
len(filtered_domains_list)

In [13]:
%%time
features_dict = defaultdict(list)
states_features_dict = defaultdict(list)
an_str = ["an_afr", "an_amr", "an_eas", "an_fin", "an_nfe", "an_oth", "an_sas"]
ac_str = ["ac_afr", "ac_amr", "ac_eas", "ac_fin", "ac_nfe", "ac_oth", "ac_sas"]

try:
    for domain_name in filtered_domains_list:

        dirfiles = !ls -t $input_path$domain_name
        filename = dirfiles[0]
        with open(input_path+domain_name+"/"+filename, 'rb') as handle:
            states_dict = pickle.load(handle)

        #Create af_adj flat dict
        states_af_adj_dict = defaultdict(list)
        for state in states_dict.keys():
            for d in states_dict[state]:
                states_af_adj_dict[state].append(d["af_adj"])

        #scale the af_dict
        states_MAF_adj_dict_scaled = defaultdict(list)
        for state in states_dict.keys():
            state_len = len(states_dict[state])
            for d in states_dict[state]:
                states_MAF_adj_dict_scaled[state].append(float(d["af_adj"]/state_len))

        #Create EMD dict
        af_adj_emd_dict = compute_pymed_emd(states_MAF_adj_dict_scaled, 1000)

        #Create a dict of conserved states
        con_states_dict = {}
        con_threshold = 0.5
        if (domain_name in hmm_prob_dict.keys()):
            for state in hmm_prob_dict[domain_name].keys():
                prob_list = hmm_prob_dict[domain_name][state]
                for i in range(len(prob_list)):
                    p = prob_list[i]
                    if (p > con_threshold):
                        major_allele = pfam_aa_order[i]
                        con_states_dict[state] = major_allele
        else:
            con_states_dict.fromkeys(states_dict.keys())
            con_states_dict.update({k : np.nan for k in con_states_dict.iterkeys()})

        #Adding states features
        for state in states_dict.keys():

            #Init counters & paramters
            maf_sum = 0
            sites_aa_alter_num = 0
            sites_snp_alter_num = 0
            sites_aa_num = len(states_dict[state])
            sites_snp_num = 3*sites_aa_num
            sites_poly_aa_num = 0 #The number of different aa in all the altered sites (most are 1)
            sites_poly_aa_several = 0

            #Rare-poly-counters
            maft_5 =  0.005
            maft_05 = 0.0005
            maft_005 = 0.00005
            rare_5_num = 0
            rare_05_num = 0
            rare_005_num = 0

            #BLOSUM62_vals
            blosum62_list = []
            weigted_blosum62_list = []

            #dn/ds counters and variables
            ref_seq = ""
            #ref_af_list = []
            Nd = 0
            Sd = 0

            #SIFT & PolyPhen couners
            sift_sum = 0
            sift_cnt = 0
            polyphen_sum = 0
            polyphen_cnt = 0

            #Populations variables
            ac_sum = [0] *len(ac_str)
            ac_sum_syn = [0] *len(ac_str)
            ac_sum_nonsyn = [0] *len(ac_str)
            an_list = [[] for i in range(len(an_str))]
            pop_maf_list = [[] for i in range(len(an_str))]
            pop_maf_syn_list = [[] for i in range(len(an_str))]
            pop_maf_nonsyn_list = [[] for i in range(len(an_str))]

            #Iterating the state dict to get properties
            for d in states_dict[state]:

                #Creating a position pseudo-ref sequence
                ref_codon = d["bp_ref"]
                ref_seq = ref_seq+ref_codon

                #Saving a list of all the ref codons prob. (most of them are close to 1)
                #ref_af = d["af_adj"]
                #ref_af_list.append(1-ref_af)

                #Calculating frequency-based N/S
                bp_af_adj_dict = d["bp_af_adj_dict"]
                for alt_codon in bp_af_adj_dict.keys():
                    alt_aa = codon_table[alt_codon]
                    #syn
                    if (alt_aa == d["aa_ref"]):
                        Sd += bp_af_adj_dict[alt_codon]
                    #Non-syn
                    else:
                        Nd += bp_af_adj_dict[alt_codon]

                if (d["af_adj"] > 0):
                    sites_aa_alter_num += 1
                    sites_snp_alter_num += len(d["an_adj"])
                    maf_sum += d["af_adj"]

                    #Number of different polymorphisms at this site
                    site_poly_num = len(d["alterations_af_adj_dict"].keys())
                    sites_poly_aa_num += site_poly_num
                    if (site_poly_num > 1):
                        sites_poly_aa_several += 1

                    #Rare poly features
                    if (d["af_adj"] < maft_005):
                        rare_005_num += 1
                        rare_05_num += 1
                        rare_5_num += 1
                    elif (d["af_adj"] < maft_05):
                        rare_05_num += 1
                        rare_5_num += 1
                    elif (d["af_adj"] < maft_5):
                        rare_5_num += 1

                    #BLOSUM62 features
                    ref = d["aa_ref"]
                    for alt in d["alterations_af_adj_dict"].keys():
                        blosum_val = blosum62_dict[ref][alt]
                        af_adj = np.mean(d["alterations_af_adj_dict"][alt])
                        blosum62_list.append(blosum_val)
                        weigted_blosum62_list.append(blosum_val*af_adj)

                    #SIFT
                    sift_list = d["SIFT"]
                    for s in sift_list:
                        if (s != ""):
                            sift_sum += float(s[s.find("(")+1:s.find(")")])
                            sift_cnt += 1
                    if (sift_cnt > 0):
                        sift_avg = sift_sum/float(sift_cnt)
                    else:
                        sift_avg = -1

                    #PolyPhen
                    polyphen_list = d["PolyPhen"]   
                    for s in polyphen_list:
                        if (s != ""):
                            polyphen_sum += float(s[s.find("(")+1:s.find(")")])
                            polyphen_cnt += 1          

                    #Saving indices of syn and non-syn bps
                    syn_idx = []
                    nonsyn_idx =[]
                    for i in range(len(d["bp_list"])):
                        ref_aa = d["aa_ref"]
                        alt_bp = d["bp_list"][i]
                        alt_aa = codon_table[alt_bp.upper()]
                        if (alt_aa == ref_aa):
                            syn_idx.append(i)
                        else:
                            nonsyn_idx.append(i)

                    #Summing the AC per population
                    for i in range(len(ac_str)):
                        ac = ac_str[i]
                        ac_sum[i] += sum(d[ac])
                        #Summing syn and non-syn seperatelly
                        ac_sum_syn[i] += sum(np.array(d[ac])[syn_idx])
                        ac_sum_nonsyn[i] += sum(np.array(d[ac])[nonsyn_idx])

                    #Averaging the AN per population, to do that, gathering all an to a list
                    for i in range(len(an_str)):
                        an = an_str[i]
                        (an_list[i]).extend(d[an])

                    #Averaging the MAF per population, to do that: gathering all maf!=0 to a list
                    for i in range(len(an_str)):
                        ac = ac_str[i]
                        an = an_str[i]
                        for j in range(len(d[ac])):
                            if (d[an][j] != 0):
                                pop_maf = d[ac][j]/float(d[an][j])
                                if (pop_maf != 0):
                                    if (j in syn_idx):
                                        pop_maf_syn_list[i].append(pop_maf)
                                    else:
                                        pop_maf_nonsyn_list[i].append(pop_maf)
                                    pop_maf_list[i].append(pop_maf)

            #Feature: EMD to all 0s distribution
            state_id = domain_name+"-"+str(state)
            features_dict[state_id].append(af_adj_emd_dict[state])

            #Feature: average MAF overall aa sites
            if (sites_aa_num == 0):
                avg_maf_overall = 0
            else:
                avg_maf_overall = maf_sum/float(sites_aa_num)
            features_dict[state_id].append(avg_maf_overall)

            #Feature: average MAF of all the altered sites
            if (sites_aa_alter_num == 0):
                avg_maf_only_altered = 0
            else:
                avg_maf_only_altered = maf_sum/float(sites_aa_alter_num)
            features_dict[state_id].append(avg_maf_only_altered)

            #Feature: number of alterations - aa level (raw and normalized by total number of matched positions)
            if (sites_aa_num == 0):
                norm_aa_alter_num = 0
            else:
                norm_aa_alter_num = sites_aa_alter_num/float(sites_aa_num)
            features_dict[state_id].append(sites_aa_alter_num)
            features_dict[state_id].append(norm_aa_alter_num)

            #Feature: number of alterations - DNA level (raw and normalized by total number of matched positions)
            if (sites_snp_num == 0):
                norm_snp_alter_num = 0
            else:
                norm_snp_alter_num = sites_snp_alter_num/float(sites_snp_num)
            features_dict[state_id].append(sites_snp_alter_num)
            features_dict[state_id].append(norm_snp_alter_num)

            #Feature: average number of poymorphisms at one site
            if (sites_aa_alter_num == 0):
                avg_poly_aa = 0
            else:
                avg_poly_aa = sites_poly_aa_num/float(sites_aa_alter_num)
            features_dict[state_id].append(avg_poly_aa)

            #Feature: fraction of altered sites with more than 1 polymorphism
            if (sites_aa_alter_num == 0):
                frac_poly_several = 1
            else:
                frac_poly_several = sites_poly_aa_several/float(sites_aa_alter_num)
            features_dict[state_id].append(frac_poly_several)

            #Feature: fraction of rare SNPs (0.5%, 0.05%, 0.005%)
            if (sites_aa_alter_num == 0):
                frac_rare_5 = 0
                frac_rare_05 = 0
                frac_rare_005 = 0
            else:
                frac_rare_5 = rare_5_num/float(sites_aa_alter_num)
                frac_rare_05 = rare_05_num/float(sites_aa_alter_num)
                frac_rare_005 = rare_005_num/float(sites_aa_alter_num)
            features_dict[state_id].append(frac_rare_5)
            features_dict[state_id].append(frac_rare_05)
            features_dict[state_id].append(frac_rare_005)

            #Feature: BLOSUM62 average
            if (len(blosum62_list) == 0):
                blosum62_avg = 0
                weigted_blosum62_avg = 0
            else:
                blosum62_avg = sum(blosum62_list)/float(len(blosum62_list))
                weigted_blosum62_avg = sum(weigted_blosum62_list)/float(len(weigted_blosum62_list))
            features_dict[state_id].append(blosum62_avg)
            features_dict[state_id].append(weigted_blosum62_avg)

            #Feature: pseudo-sequence dN/dS        
            (N,S) = seq_ns(ref_seq) #Refrence expected syn/nonsyn per site
            if (N == 0): 
                PN = 0
            else:
                PN = Nd/float(N) #Proportion of nonsyn
            if (S == 0):
                PS = 0
            else:
                PS = Sd/float(S) #Proportion of syn

            #num of nonsyn substitutions per syn site
            dN = -0.75 * (np.log(1-4*PN/float(3)))
            #num of syn substitutions per nonsyn site
            dS = -0.75 * (np.log(1-4*PS/float(3)))

            if (dN ==0 or dS == 0):
                dN_dS = 1 #There isn't enough information to calculate dN/dS
            else:
                dN_dS = dN/dS

            features_dict[state_id].append(dN_dS)

            #Feature: Pfam HMM-emission prob.
            state_emiss_prob = max(hmm_prob_dict[domain_name][state])
            features_dict[state_id].append(state_emiss_prob)

            #Feature: Binding-score
            #if (state in binding_all_scores_dict[domain_name]["states"]):
            #    score_idx = (binding_all_scores_dict[domain_name]["states"]).index(state)
            #    binding_score = binding_all_scores_dict[domain_name]["scores"][score_idx]
            #else:
            #    binding_score = 0
            #features_dict[state_id].append(binding_score)

            #Feature: SIFT score
            if (sift_cnt > 0):
                sift_avg = sift_sum/float(sift_cnt)
            else:
                sift_avg = -1
            features_dict[state_id].append(sift_avg)

            #Feature: Polyphen score
            if (polyphen_cnt > 0):
                polyphen_avg = polyphen_sum/float(polyphen_cnt)
            else:
                polyphen_avg = -1
            features_dict[state_id].append(polyphen_avg)

            #Feature: state type (0 = Neutral)
            #state_type = "0" #Initializing as "Neutral"
            #if (state in con_states_dict.keys()):
            #    state_type = "2" #Structural
            #if (state in binding_scores_dict[domain_name]["states"]):
            #    state_type = "1" #Binding
            #features_dict[state_id].append(state_type)

            #Feature: populations ac sum
            for i in range(len(ac_str)):
                features_dict[state_id].append(ac_sum[i])

            #Feature: populations ac syn sum
            for i in range(len(ac_str)):
                features_dict[state_id].append(ac_sum_syn[i])

            #Feature: populations ac nonsyn sum
            for i in range(len(ac_str)):
                features_dict[state_id].append(ac_sum_nonsyn[i])

            #Feature: populations an avg        
            for i in range(len(an_str)):
                if (len(an_list[i]) == 0):
                    an_avg = 0
                else:
                    an_avg = np.average(an_list[i])
                features_dict[state_id].append(an_avg)

            #Feature: populations total maf avg
            for i in range(len(an_str)):
                if (len(pop_maf_list[i]) == 0):
                    avg_pop_maf = 0
                else:
                    avg_pop_maf = np.average(pop_maf_list[i])
                features_dict[state_id].append(avg_pop_maf)

            #Feature: populations syn maf avg
            for i in range(len(an_str)):
                if (len(pop_maf_syn_list[i]) == 0):
                    avg_pop_maf_syn = 0
                else:
                    #print "pop_maf_syn_list not empty"
                    #raise GetOutOfLoop
                    avg_pop_maf_syn = np.average(pop_maf_syn_list[i])
                features_dict[state_id].append(avg_pop_maf_syn)

            #Feature: populations non-syn maf avg
            for i in range(len(an_str)):
                if (len(pop_maf_nonsyn_list[i]) == 0):
                    avg_pop_maf_nonsyn = 0
                else:
                    avg_pop_maf_nonsyn = np.average(pop_maf_nonsyn_list[i])
                features_dict[state_id].append(avg_pop_maf_nonsyn)

            #Feature: domain name
            features_dict[state_id].append(domain_name)
        
except GetOutOfLoop:
    pass
    
    #print "Finished "+domain_name

CPU times: user 25min 30s, sys: 1min 43s, total: 27min 13s
Wall time: 18min 1s


In [12]:
np.average(pop_maf_syn_list[i])

0.00024265954865323951

In [2]:
class GetOutOfLoop( Exception ):
    pass

In [23]:
pop_maf_list

[[9.688044952528579e-05, 9.646922631680494e-05],
 [],
 [0.0001201634222542658],
 [],
 [5.998440405494571e-05,
  1.502494140272853e-05,
  6.0457664520419574e-05,
  1.618908855431439e-05],
 [],
 []]

In [22]:
pop_maf_syn_list

[[], [], [], [], [], [], []]

In [19]:
#Exporting to data-frames table
domains_features_df = pd.DataFrame.from_dict(features_dict,orient='index')
domains_features_df.columns = ["EMD", "avg_maf_all", "avg_maf_altered", "alter_num_aa", "alter_num_aa_norm", 
                               "alter_num_dna", "alter_num_dna_norm", "avg_poly", "frac_poly_several", "rare_poly_0.5", "rare_poly_0.05", "rare_poly_0.005", 
                               "BLOSUM_avg", "weighted_BLOSUM_avg", "pseudo_dNdS","Pfam_emission_prob", "SIFT", "PolyPhen", 
                               "ac_afr", "ac_amr", "ac_eas", "ac_fin", "ac_nfe", "ac_oth", "ac_sas", 
                               "ac_syn_afr", "ac_syn_amr", "ac_syn_eas", "ac_syn_fin", "ac_syn_nfe", "ac_syn_oth", "ac_syn_sas", 
                               "ac_nonsyn_afr", "ac_nonsyn_amr", "ac_nonsyn_eas", "ac_nonsyn_fin", "ac_nonsyn_nfe", "ac_nonsyn_oth", "ac_nonsyn_sas", 
                               "an_afr", "an_amr", "an_eas", "an_fin", "an_nfe", "an_oth", "an_sas", 
                               "maf_afr", "maf_amr", "maf_eas", "maf_fin", "maf_nfe", "maf_oth", "maf_sas",
                               "maf_syn_afr", "maf_syn_amr", "maf_syn_eas", "maf_syn_fin", "maf_syn_nfe", "maf_syn_oth", "maf_syn_sas",
                               "maf_nonsyn_afr", "maf_nonsyn_amr", "maf_nonsyn_eas", "maf_nonsyn_fin", "maf_nonsyn_nfe", "maf_nonsyn_oth", "maf_nonsyn_sas","domain_name"]
domains_features_df = domains_features_df.sort_index()

#Save to file
domains_features_df.to_csv(curr_dir[0]+"/populations/binding10_pop.csv", sep='\t')

In [21]:
domains_features_df

,EMD,avg_maf_all,avg_maf_altered,alter_num_aa,alter_num_aa_norm,alter_num_dna,alter_num_dna_norm,avg_poly,frac_poly_several,rare_poly_0.5,...,maf_syn_oth,maf_syn_sas,maf_nonsyn_afr,maf_nonsyn_amr,maf_nonsyn_eas,maf_nonsyn_fin,maf_nonsyn_nfe,maf_nonsyn_oth,maf_nonsyn_sas,domain_name
2OG-FeII_Oxy_3-1,1.592089e-11,5.284545e-06,0.000029,2,0.181818,3,0.090909,1.500000,0.500000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-10,4.562667e-12,3.018000e-06,0.000011,3,0.272727,4,0.121212,1.333333,0.333333,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-11,5.005620e-11,1.055370e-05,0.000026,4,0.400000,7,0.233333,1.250000,0.250000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-12,1.594177e-11,4.429500e-06,0.000022,2,0.200000,2,0.066667,1.000000,0.000000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-13,1.309807e-10,1.454900e-05,0.000040,4,0.363636,5,0.151515,1.000000,0.000000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-14,1.932214e-10,1.721000e-05,0.000038,5,0.454545,8,0.242424,1.200000,0.200000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-15,2.916725e-10,1.864264e-05,0.000051,4,0.363636,5,0.151515,1.000000,0.000000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-16,7.910753e-12,5.254727e-06,0.000014,4,0.363636,4,0.121212,1.000000,0.000000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-17,5.607287e-13,7.488182e-07,0.000008,1,0.090909,1,0.030303,1.000000,0.000000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
2OG-FeII_Oxy_3-18,1.184363e-11,5.268091e-06,0.000012,5,0.454545,5,0.151515,1.000000,0.000000,1.00,...,0,0,0,0,0,0,0,0,0,2OG-FeII_Oxy_3
